## Instruction

Attached is a clean dataset that shows performance of 25 psychiatric providers across 5 metrics over the last 12 months. This data is fake but represents a realistic scenario. 

Please take this data to do three things: 

`Task 1`. Get to know the data better by writing 2 SQL queries to answer the following 
questions: 

   * Identify outlier providers who are consistently underperforming (defined 
as missing 3 out of 5 targets) over the last 3 months.
   * Identify the top 5 providers who have made the most improvement over 
time to their MIC utilization rate. 

`Task 2`. Write a script that can take data structured like this and generate a personalized Google sheet for each provider that can be shared with them on a monthly basis so they can understand their performance across each of these metrics through 
time against targets 

`Task 3`. Write a document for the psych management team that shows overall 
performance across these metrics. Highlight trends, diagnose gaps goals, and 
identify recommendations you would make to close them (could be hypotheses 
you’d want to test if you had more data as well) 

In [14]:
# required packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [15]:
# Load the source data
df_metric_score_by_provider_monthly = pd.read_excel("../data/Rula Insights Manager Case Data.xlsx")

In [16]:
# Review data structure.
print(df_metric_score_by_provider_monthly.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Provider       709 non-null    object        
 1   month_year     709 non-null    datetime64[ns]
 2   metric_score   709 non-null    float64       
 3   metric_source  709 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 22.3+ KB
None


In [17]:
df_metric_score_by_provider_monthly.head(10)

,Provider,month_year,metric_score,metric_source
0,Provider1,2024-01-01,4.700000,average_responses_agg_month
1,Provider1,2024-02-01,4.500000,average_responses_agg_month
2,Provider1,2024-03-01,4.428571,average_responses_agg_month
3,Provider1,2024-04-01,4.636364,average_responses_agg_month
4,Provider1,2024-05-01,4.411765,average_responses_agg_month
5,Provider1,2024-06-01,4.285714,average_responses_agg_month
6,Provider1,2024-07-01,5.000000,average_responses_agg_month
7,Provider1,2024-08-01,4.526316,average_responses_agg_month
8,Provider10,2023-12-01,4.500000,average_responses_agg_month
9,Provider10,2024-01-01,4.636364,average_responses_agg_month


## Data Load and Quality Check

* Load the provided data
* Check assumptions on the data holds true. Assumptions are:
  1. All required metrics are available in `metric_source` column.
  2. Each row is unique per combination of (`provider`, `metric_source`, `month_year`). 
  3. For given `provider` and `metric_source`, there isn't a gap between months.

### Check 01: Metrics are available
Let's first check to see if all five metrics and 25 providers exists in the given dataset.

In [18]:
df_metric_score_by_provider_monthly['metric_source'].unique(),len(df_metric_score_by_provider_monthly['metric_source'].unique())

(array(['average_responses_agg_month', 'caseloads_months',
        'chart_review_months', 'documentation_rates_months',
        'mic_utilization_rate_months'], dtype=object),
 5)

In [19]:
df_metric_score_by_provider_monthly['Provider'].unique(), len(df_metric_score_by_provider_monthly['Provider'].unique())

(array(['Provider1', 'Provider10', 'Provider11', 'Provider12',
        'Provider13', 'Provider14', 'Provider15', 'Provider16',
        'Provider17', 'Provider18', 'Provider19', 'Provider2',
        'Provider20', 'Provider21', 'Provider22', 'Provider23',
        'Provider24', 'Provider27', 'Provider3', 'Provider40', 'Provider7',
        'Provider8', 'Provider9', 'Provider32', 'Provider41'], dtype=object),
 25)

### Check 02: Row Uninuqess
Is each row in a given dataset unique, given provider, metric_source, and month_year?

In [20]:
result = df_metric_score_by_provider_monthly\
    .groupby(['Provider', 'metric_source', 'month_year'])\
    .agg({'metric_score': 'count'})\
    .query('metric_score > 1')

print(result)

Empty DataFrame
Columns: [metric_score]
Index: []


### Check 03: No Gaps in Date Range
Are metrics are provided consistently without gaps? 

The table below unfortunately shows that there are some month where there is a gap in date range.

In [21]:
# Ensure the dataframe is sorted by provider_id, metric_source, and month_year
df_metric_score_by_provider_monthly = df_metric_score_by_provider_monthly\
    .sort_values(by=['Provider', 'metric_source', 'month_year'], ascending=[True, True, True])\
    .reset_index(drop=True)

# Calculate the difference in month_year for each row compared to the previous row within each partition
df_metric_score_by_provider_monthly['month_year_diff'] = df_metric_score_by_provider_monthly\
    .groupby(['Provider', 'metric_source'])['month_year']\
    .diff().apply(lambda x: x / pd.Timedelta(days=30)).astype('float').round().astype('Int64')

df_metric_score_by_provider_monthly[df_metric_score_by_provider_monthly["month_year_diff"] > 1]

,Provider,month_year,metric_score,metric_source,month_year_diff
21,Provider1,2024-03-01,1.000000,chart_review_months,2
104,Provider11,2024-05-01,1.000000,chart_review_months,2
106,Provider11,2024-08-01,1.000000,chart_review_months,2
142,Provider12,2024-08-01,1.000000,chart_review_months,2
178,Provider13,2024-05-01,1.000000,chart_review_months,2
215,Provider14,2024-07-01,0.666667,chart_review_months,2
247,Provider15,2024-05-01,1.000000,chart_review_months,2
249,Provider15,2024-08-01,0.800000,chart_review_months,2
280,Provider16,2024-07-01,0.000000,chart_review_months,2
307,Provider17,2024-06-01,1.000000,chart_review_months,2


The above table shows that there are unfortunately quite a bit of gaps in date ranges.

Let's take a look at Provider #3's `chart_review_months`. You can see that it is missing 2024-01, 02, 04, and 05.
We will address his issue in Data Cleaning step.

In [22]:
# Spotcheck Assumption #3
df_metric_score_by_provider_monthly.query("(Provider == 'Provider3') & (metric_source == 'chart_review_months')")

,Provider,month_year,metric_score,metric_source,month_year_diff
541,Provider3,2023-12-01,1.000000,chart_review_months,<NA>
542,Provider3,2024-03-01,1.000000,chart_review_months,3
543,Provider3,2024-07-01,0.833333,chart_review_months,4
544,Provider3,2024-08-01,1.000000,chart_review_months,1


### Check 04: Metric value (non-nulls and expected range)
All metrics are within defined bounds, which are:
* average_responses_agg_month (between 0 and 4)
* caseloads_month (between 0 and 1000)
* chart_review_months (between 0 and 1)
* documentation_rates_months (between 0 and 1)
* mic_utilization_rate_months (between 0 and 1)

In [23]:
def check_data_quality(df, value_ranges, ignore_columns=None):
    """
    Check data quality for a dataframe.
    
    Parameters:
    df (pd.DataFrame): The dataframe to check.
    value_ranges (dict): A dictionary where keys are column names and values are lists containing [min, max] range.
    ignore_columns (list): A list of columns to ignore for checking.
    
    Returns:
    dict: A dictionary with columns as keys and a list of issues found as values.
    """
    if ignore_columns is None:
        ignore_columns = []
    
    issues = {}
    
    # Check for null values
    null_issues = df.isnull().sum()
    for col, null_count in null_issues.items():
        if null_count > 0 and col not in ignore_columns:
            issues[col] = issues.get(col, []) + [f"Null values: {null_count}"]
    
    # Check for values out of range
    for col, (min_val, max_val) in value_ranges.items():
        if col in df.columns and col not in ignore_columns:
            out_of_range = df[(df[col] < min_val) | (df[col] > max_val)]
            if not out_of_range.empty:
                issues[col] = issues.get(col, []) + [f"Values out of range: {len(out_of_range)}"]
    
    return issues


# Define the value ranges for each column
value_ranges = {
    'average_responses_agg_month': [0, 4],
    'caseloads_month': [0, 1000],
    'chart_review_months': [0, 1],
    'documentation_rates_months': [0, 1],
    'mic_utilization_rate_months': [0, 1]
}

# Specify columns to ignore
ignore_columns = ['month_year_diff']

# Check data quality
data_quality_issues = check_data_quality(df_metric_score_by_provider_monthly, value_ranges, ignore_columns)
data_quality_issues

{}